In [1]:
import pandas as pd
import geopandas as gpd
import joblib

model = joblib.load('../../models/model_forest.pkl')
scaler = joblib.load('../../models/scaler.pkl')

In [2]:
gdf_ = gpd.read_file('../../data/processed/building/building_240417.geojson')

In [4]:
gdf_.drop(columns=['geometry']).to_csv('./building_0418.csv', index=False)

In [21]:
df_building = gpd.read_file('../../data/processed/building/building_240417.geojson')

In [22]:
df_building_copy = df_building.copy()

In [23]:
df_building_copy = df_building_copy.rename(columns={'distance_to_park':'dist_park',
                            'retail_within_450ft':'retail_450',
                            'distance_from_station(ft)':'dist_station',
                            'residential_within_450ft':'residential_450',
                            'distance_to_school':'dist_school',
                            'office_within_450ft':'office_450'})

In [24]:
columns = ['dist_park',
           'retail_450',
           'retail_area',
           'dist_station',
           'residential_450',
           'idw_atvc_mean',
           'food_1000',
           'food_800',
           'idw_aadt_mean',
           'residential_area',
           'dist_school',
           'food_400',
           'office_450',
           'food_100']

In [25]:
X_buidling = df_building_copy.loc[:, columns]
X_buidling = scaler.transform(X_buidling)
X_buidling = pd.DataFrame(X_buidling, columns=columns)
y_building = [values[1] for values in model.predict_proba(X_buidling)]

In [26]:
df_building.loc[:,'probability'] = y_building

In [28]:
df_building.to_file('../../data/processed/building/building_240417_probability.geojson', driver='GeoJSON')

In [30]:
df_building.sort_values(by='probability', ascending=False).head(10).loc[:,['bin','probability']]

,bin,probability
3742,1063584,0.700462
8500,1082666,0.698784
11016,1085800,0.693452
2950,1088298,0.690602
11982,1020543,0.688413
32374,1088320,0.687427
16868,1085400,0.686722
14064,1001137,0.686369
1472,1087577,0.684961
13619,1085799,0.684014
